In [1]:
import os
import pickle
import repackage
repackage.up()
import os.path as osp
from nuscenes.nuscenes import NuScenes
from tqdm import tqdm
import pandas as pd
import numpy as np
from datasets.nuscenes import NuScenesMultipleRadarMultiSweeps
from utils.labelling import get_sps_labels
from utils.transforms import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sps_df = pd.read_json('../sps_nuscenes_more_matches_df.json')
# sps_df = pd.read_json('../nuscenes_scenes_df.json')
sps_df.head()

,scene_name,first_frame_datetime,days_since_first_recording,hours_since_first_recording,month,split,closest_scenes,closest_scenes_data
121,scene-0161,1526922443042,0,0.000000,May,trainval,[scene-0218],{'scene-0218': {'scene_token': 'febc1800b9ed43...
122,scene-0162,1526922463034,0,0.005553,May,trainval,"[scene-0665, scene-0218, scene-0219]",{'scene-0665': {'scene_token': '45275e709d4a4b...
123,scene-0163,1526922483050,0,0.011113,May,trainval,"[scene-0075, scene-0511, scene-0332, scene-045...",{'scene-0075': {'scene_token': '01c3f5e3995640...
124,scene-0164,1526922518041,0,0.020833,May,trainval,"[scene-0513, scene-0091]",{'scene-0513': {'scene_token': 'e333874a12d64a...
125,scene-0165,1526922555077,0,0.031121,May,trainval,"[scene-0092, scene-0333, scene-0265, scene-0062]",{'scene-0092': {'scene_token': '7365495b744646...


In [ ]:
ref_frame = None
num_sweeps = 5
ref_sensor = None
apply_dpr = False
filter_points = False
dpr_thresh = 0.75

data_dir = "/shared/data/nuScenes/"
sps_maps_dir = "../output_sw5-dpr0.15-r1_combined_maps/labelled_maps/"

In [ ]:

sensors = ["RADAR_FRONT", "RADAR_FRONT_LEFT", "RADAR_FRONT_RIGHT", "RADAR_BACK_LEFT", "RADAR_BACK_RIGHT"]
versions = {'trainval': 'v1.0-trainval', 'test': 'v1.0-test'}
nuscenes_exp = {
    vname : NuScenes(dataroot=data_dir, version=version, verbose=False)\
    for vname,version in versions.items()
}

In [ ]:
def extract_data(dataset):
    num_frames = len(dataset)
    local_poses = dataset.local_poses
    global_poses = dataset.global_poses
    ego_timestamps = dataset.timestamps
    all_data = []

    for i in range(num_frames):
        frame_dict = {}
        (pointclouds, sps_scores), calibs, sensors, timestamps = dataset[i]

        for sensor, calib, ts, pcl, scores in zip(sensors, calibs, timestamps, pointclouds, sps_scores):
            frame_dict[sensor] = {
                'calib' : calib,
                'timestamp': ts,
                'pointcloud': pcl,
                'stability_scores': scores
            }
        frame_dict['ego_pose'] = global_poses[i]
        frame_dict['ego_local_pose'] = local_poses[i]
        frame_dict['ego_timestamp'] = ego_timestamps[i]
        all_data.append(frame_dict)
    return all_data

In [ ]:
row = sps_df.iloc[0]
ref_scene_name = row['scene_name']
ref_split = row['split']
closest_scenes = row['closest_scenes_data']
seq = int(ref_scene_name.split("-")[-1])


dataset_sequence = NuScenesMultipleRadarMultiSweeps(
    data_dir=data_dir,
    nusc=nuscenes_exp[ref_split],
    sequence=seq,
    sensors=sensors,
    nsweeps=num_sweeps,
    ref_frame=ref_frame,
    ref_sensor=ref_sensor,
    sps_thresh=0.0,
    return_sps_scores=True,
    sps_labels_dir=sps_maps_dir,
    apply_dpr=apply_dpr,
    filter_points=filter_points,
    ransac_threshold=dpr_thresh,
    reformat_pcl=False

)

data = extract_data(dataset_sequence)

In [ ]:
print(seq)

In [ ]:
data[0]['ego_timestamp']

In [ ]:
data[0]['ego_pose']

In [ ]:
data[0].keys()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


def plot_pointclouds(sensor_data, ego_pose):
    """
    Plots the pointclouds from all sensors in the global frame with color coding based on stability scores.
    """
    all_points = []
    all_scores = []

    if type(sensor_data) == list:
        for sd, pose in zip(sensor_data, ego_pose):
            for sensor, data in sd.items():
                if sensor == 'ego_pose' or sensor == 'ego_timestamp' or sensor == 'ego_local_pose':
                    continue
                calib_matrix = data['calib']
                pointcloud = data['pointcloud']
                stability_scores = data['stability_scores']
                
                # Transform points from sensor to ego vehicle frame
                ego_frame_points = transform_doppler_points(calib_matrix, pointcloud)
                
                # Transform points from ego vehicle frame to global frame
                global_frame_points = transform_doppler_points(pose, ego_frame_points)
                
                # Append points and their color based on individual stability scores
                all_points.append(global_frame_points)
                all_scores.append(stability_scores)
    
    all_points = np.vstack(all_points)
    all_scores = np.concatenate(all_scores)

    # Create scatter plot
    fig = plt.figure(figsize=(16, 16))

    ax = fig.add_subplot(111)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    sc = ax.scatter(all_points[:, 0], all_points[:, 1], c=all_scores, cmap='RdYlGn', s=0.5)
    
    plt.title('Pointclouds from All Sensors in Global Frame')
    plt.colorbar(sc)
    plt.show()

In [ ]:
plot_pointclouds(data[:5], [d['ego_pose'] for d in data[:5]])

In [ ]:
plot_pointclouds(data[:5], [d['ego_local_pose'] for d in data[:5]])

In [ ]:
import os
def save_sensor_data(data, sequence_name, base_dir):
    """
    Save the concatenated sensor data to disk in the specified structure.
    
    Parameters:
    - data: The list of sensor data dictionaries.
    - sequence_name: The name of the sequence for which data is being saved.
    - base_dir: The base directory where the data will be saved.
    """
    # Create directories
    sequence_dir = os.path.join(base_dir, "sequence", sequence_name)
    scans_dir = os.path.join(sequence_dir, "scans")
    poses_dir = os.path.join(sequence_dir, "poses")
    local_poses_dir = os.path.join(sequence_dir, "local_poses")
    labels_dir = os.path.join(sequence_dir, "labels")
    map_transform_dir = os.path.join(sequence_dir, "map_transform")

    os.makedirs(scans_dir, exist_ok=True)
    os.makedirs(poses_dir, exist_ok=True)
    os.makedirs(local_poses_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(map_transform_dir, exist_ok=True)

    # Process each element in the data list
    for idx, sensor_data in enumerate(data):
        combined_pointclouds = []
        combined_sps_scores = []
        for sensor in sensor_data:
            if sensor == 'ego_pose' or sensor == 'ego_timestamp' or sensor == 'ego_local_pose':
                continue

            calib_matrix = sensor_data[sensor]['calib']
            pointclouds = sensor_data[sensor]['pointcloud']
            
            transformed_points = transform_doppler_points(calib_matrix, pointclouds)
            combined_pointclouds.append(transformed_points)
            combined_sps_scores.append(sensor_data[sensor]['stability_scores'])

        combined_pointcloud = np.vstack(combined_pointclouds)
        combined_sps_scores = np.hstack(combined_sps_scores)

        assert(combined_sps_scores.shape[0] == combined_pointcloud.shape[0])
        
        # Save combined pointcloud and ego pose
        ego_timestamp = str(np.mean(np.array(sensor_data['ego_timestamp'])))
        scan_file = os.path.join(scans_dir, f"{ego_timestamp}.npy")
        pose_file = os.path.join(poses_dir, f"{ego_timestamp}.txt")
        local_pose_file = os.path.join(local_poses_dir, f"{ego_timestamp}.txt")
        label_file = os.path.join(labels_dir, f"{ego_timestamp}.npy")

        np.save(scan_file, combined_pointcloud)
        np.savetxt(pose_file, sensor_data['ego_pose'], delimiter=',')
        np.savetxt(local_pose_file, sensor_data['ego_local_pose'], delimiter=',')
        np.save(label_file, combined_sps_scores)

        # Save a dummy map_transform file if needed
        map_transform_file = os.path.join(map_transform_dir, "map_transform.txt")
        if not os.path.exists(map_transform_file):
            dummy_transform = np.eye(4)
            np.savetxt(map_transform_file, dummy_transform, delimiter=',')

In [ ]:
import shutil
base_dir = "/home/umair/workspace/radar_sps_datasets/nuscenes"

In [ ]:

if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
    os.mkdir(base_dir)
    
src_maps_dir = "/home/umair/workspace/radar_auto_labeler/output_sw5-dpr0.15-r2_combined_maps/labelled_maps/"
dst_maps_dir = os.path.join(base_dir, "maps")
shutil.copytree(src_maps_dir, dst_maps_dir)

In [ ]:
## Main loop

for i,row in tqdm(sps_df.iterrows(), total=len(sps_df)):
    ref_scene_name = row['scene_name']
    ref_split = row['split']
    closest_scenes = row['closest_scenes_data']
    seq = int(ref_scene_name.split("-")[-1])


    # dataset_sequence = NuScenesMultipleRadarMultiSweeps(
    #     data_dir=data_dir,
    #     nusc=nuscenes_exp[ref_split],
    #     sequence=seq,
    #     sensors=sensors,
    #     nsweeps=num_sweeps,
    #     ref_frame=ref_frame,
    #     ref_sensor=ref_sensor,
    #     sps_thresh=0.0,
    #     return_sps_scores=True,
    #     sps_labels_dir=sps_maps_dir,
    #     apply_dpr=False,
    #     filter_points=False,
    #     ransac_threshold=-1,
    #     reformat_pcl=False

    # )

    # data = extract_data(dataset_sequence)
    # save_sensor_data(data, ref_scene_name, base_dir)

    for matched_scene, data in closest_scenes.items():
        print(matched_scene)
        print(data)
        assert(0)
        ds = NuScenesMultipleRadarMultiSweeps(
            data_dir=data,
            nusc=nuscenes_exp[data['split']],
            sequence=int(matched_scene.split("-")[-1]),
            sensors=sensors,
            nsweeps=num_sweeps,
            ref_frame=ref_frame,
            ref_sensor=ref_sensor,
            apply_dpr=False,
            filter_points=False,
            ransac_threshold=-1,
            reformat_pcl=False
        )
        data = extract_data(dataset_sequence)
        save_sensor_data(data, ref_scene_name, base_dir)


In [ ]:
def combine_maps(sps_maps_dir):
    """
    Combines all .asc maps from the given directory into a single map and writes it to disk.
    
    Parameters:
    - sps_maps_dir: The directory containing the .asc map files.
    """
    combined_data = []

    # Iterate through all .asc files in the directory
    for file_name in os.listdir(sps_maps_dir):
        if file_name.endswith(".asc"):
            file_path = os.path.join(sps_maps_dir, file_name)
            # Read the .asc file
            data = np.loadtxt(file_path, skiprows=1)
            combined_data.append(data)
    
    # Combine all the data into a single array
    combined_data = np.vstack(combined_data)
    
    # Determine the output file path one level up from sps_maps_dir
    parent_dir = os.path.abspath(os.path.join(sps_maps_dir, os.pardir))
    output_file = os.path.join(parent_dir, "combined_map.asc")
    
    # Write the combined data to a new .asc file
    np.savetxt(output_file, combined_data, fmt="%.6f")
    print(f"Combined map saved to {output_file}")
    return combined_data

In [ ]:
full_map = combine_maps(sps_maps_dir)

In [ ]:
full_map = np.load('/home/umair/workspace/radar_auto_labeler/notebooks/boston-seaport_sps.asc',)

In [ ]:
import matplotlib.pyplot as plt
# Create scatter plot
fig = plt.figure(figsize=(16, 16))
ax = fig.add_subplot(111)
sc = ax.scatter(full_map[:, 0], full_map[:, 1], c=full_map[:,-1], cmap='RdYlGn', s=0.01)

ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(sc)
plt.title('Pointclouds from All Sensors in Global Frame')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Create scatter plot
sps_only_map = full_map[full_map[:,-1] > 0.0]

fig = plt.figure(figsize=(16, 16))
ax = fig.add_subplot(111)
sc = ax.scatter(sps_only_map[:, 0], sps_only_map[:, 1], c=sps_only_map[:,-1], cmap='RdYlGn', s=0.01)

ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(sc)
plt.title('Pointclouds from All Sensors in Global Frame')
plt.show()

### Create Splits and update SPS/RIT training configs

In [3]:
# Load the dataframe from the JSON file
import yaml

# Shuffle the dataframe to ensure randomness
df_shuffled = sps_df.sample(frac=1).reset_index(drop=True)

# Split the data into train (80%), val (10%), test (10%)
train_split = int(0.8 * len(df_shuffled))
val_split = int(0.9 * len(df_shuffled))

train_df = df_shuffled.iloc[:train_split]
val_df = df_shuffled.iloc[train_split:val_split]
test_df = df_shuffled.iloc[val_split:]

In [4]:
len(train_df), len(val_df), len(test_df)

(126, 16, 16)

In [5]:
def populate_sps_config(train_df, val_df, test_df, config_file, new_config_file):
    """
    Populates the SPS config YAML file based on the train/val/test splits from the dataframe.
    
    Parameters:
    train_df (pd.DataFrame): DataFrame containing training scene information.
    val_df (pd.DataFrame): DataFrame containing validation scene information.
    test_df (pd.DataFrame): DataFrame containing test scene information.
    config_file (str): Path to the original YAML configuration file.
    new_config_file (str): Path where the updated configuration will be saved.
    """
    # Load the YAML config file
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)

    # Function to update the config file with scene names and closest scenes
    def update_config(df_split, split_type):
        scene_names = df_split['scene_name'].tolist()
        closest_scenes_list = df_split['closest_scenes'].tolist()
        
        config['DATA']['MAPS'][split_type] = scene_names
        config['DATA']['SPLIT'][split_type] = [scene for closest_scenes in closest_scenes_list for scene in closest_scenes]

    # Update config for train, val, and test splits
    update_config(train_df, 'TRAIN')
    update_config(val_df, 'VAL')
    update_config(test_df, 'TEST')

    print('train: ', len(config['DATA']['SPLIT']['TRAIN']))
    print('val: ', len(config['DATA']['SPLIT']['VAL']))
    print('test: ', len(config['DATA']['SPLIT']['TEST']))

    # Save the updated config to a new YAML file
    # with open(new_config_file, 'w') as file:
        # yaml.dump(config, file)

    # print(f"Updated SPS config file saved as {new_config_file}")

In [6]:
def populate_rit_config(train_df, val_df, test_df, config_file, new_config_file):
    """
    Populates the RIT config YAML file based on the train/val/test splits from the dataframe.
    
    Parameters:
    train_df (pd.DataFrame): DataFrame containing training scene information.
    val_df (pd.DataFrame): DataFrame containing validation scene information.
    test_df (pd.DataFrame): DataFrame containing test scene information.
    config_file (str): Path to the original YAML configuration file.
    new_config_file (str): Path where the updated configuration will be saved.
    """
    # Load the YAML config file
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)

    # Helper function to convert scene names to integers
    def convert_scene_names(df_split):
        # Remove 'scene-' and cast to int
        scene_names = df_split['scene_name'].apply(lambda x: int(x.replace('scene-', ''))).tolist()
        closest_scenes_list = df_split['closest_scenes'].apply(lambda x: [int(scene.replace('scene-', '')) for scene in x]).tolist()
        return scene_names, [scene for closest_scenes in closest_scenes_list for scene in closest_scenes]

    # Convert train, val, and test scene names
    train_maps, train_splits = convert_scene_names(train_df)
    val_maps, val_splits = convert_scene_names(val_df)
    test_maps, test_splits = convert_scene_names(test_df)

    # Update the config with the converted scene names
    config['maps']['train'] = train_maps
    config['maps']['valid'] = val_maps
    config['maps']['test'] = test_maps

    config['split']['train'] = train_splits
    config['split']['valid'] = val_splits
    config['split']['test'] = test_splits

    print('train: ', len(train_splits))
    print('val: ', len(val_splits))
    print('test: ', len(test_splits))

    # Save the updated config to a new YAML file
    # with open(new_config_file, 'w') as file:
        # yaml.dump(config, file)

    # print(f"Updated RIT config file saved as {new_config_file}")

In [7]:
# Populate SPS config with pre-split dataframes
sps_config_file = "/home/umair/workspace/SPS/config/config.yaml"
rit_config_file = "/home/umair/workspace/rit-master/config/radar_scenes/radar_mapping.yaml"

populate_sps_config(train_df, val_df, test_df, sps_config_file, 'config.yaml')

# Populate RIT config with the same splits
populate_rit_config(train_df, val_df, test_df, rit_config_file, 'radar_mapping.yaml')

train:  295
val:  40
test:  39
train:  295
val:  40
test:  39
